In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import py7zlib
from io import StringIO
import dask.bag as db
from dask.distributed import Client
client = Client()
# check status at: http://localhost:8787/status

In [2]:
data_dir = "data/ares"

In [3]:
vr_full_name = pd.read_csv(f"{data_dir}/ares_ciselnik_VR.csv", sep=';', header=None).set_index(0)[1]
vr_full_name

0
NR                              Nadační rejstřík
OR                             Obchodní rejstřík
ROPS     Rejstřík obecně prospěšných společností
RSVJ    Rejstřík společenství vlastníků jednotek
RU                               Rejstřík ústavů
SR                             Spolkový rejstřík
Name: 1, dtype: object

In [37]:
ico_fn = "ares_seznamIC_VR.csv"
with open(f"{data_dir}/{ico_fn}.7z", 'rb') as file:
    archive = py7zlib.Archive7z(file)

    csv = StringIO(archive.getmember(ico_fn).read().decode())
    
    ico = pd.read_csv(csv, sep=';', header=None, names=['ICO', 'date_processed'])

ico.head()

,ICO,date_processed
0,108,24.10.2015
1,124,24.10.2015
2,205,12.01.2017
3,248,24.10.2015
4,256,24.10.2015
